<div id="singlestore-header" style="display: flex; background-color: rgba(209, 153, 255, 0.25); padding: 5px;">
    <div id="icon-image" style="width: 90px; height: 90px;">
        <img width="100%" height="100%" src="https://raw.githubusercontent.com/singlestore-labs/spaces-notebooks/master/common/images/header-icons/vector-circle.png" />
    </div>
    <div id="text" style="padding: 5px; margin-left: 10px;">
        <div id="badge" style="display: inline-block; background-color: rgba(0, 0, 0, 0.15); border-radius: 4px; padding: 4px 8px; align-items: center; margin-top: 6px; margin-bottom: -2px; font-size: 80%">SingleStore Notebooks</div>
        <h1 style="font-weight: 500; margin: 8px 0 0 4px;">Hybrid Search</h1>
    </div>
</div>

<div class="alert alert-block alert-warning">
    <b class="fa fa-solid fa-exclamation-circle"></b>
    <div>
        <p><b>Note</b></p>
        <p> This notebook can be run on a Free Starter Workspace. To create a Free Starter Workspace navigate to <tt>Start</tt> using the left nav. You can also use your existing Standard or Premium workspace with this Notebook. </p>
    </div>
</div>

*Source*: [OpenAI Cookbook](https://github.com/openai/openai-cookbook/blob/main/examples/data/AG_news_samples.csv)

Hybrid search integrates both keyword-based search and semantic search in order to combine the strengths of both and provide users with a more comprehensive and efficient search experience. This notebook is an example on how to perform hybrid search with SingleStore's database and notebooks.

## Setup
Let's first download the libraries necessary.

In [7]:
!pip install wget --quiet
!pip install openai==1.3.3 --quiet

In [8]:
import json
import os
import pandas as pd
import wget

In [9]:
# Import the library for vectorizing the data (Up to 2 minutes)
!pip install sentence-transformers --quiet

from sentence_transformers import SentenceTransformer

model = SentenceTransformer('flax-sentence-embeddings/all_datasets_v3_mpnet-base')

## Import data from CSV file
This csv file holds the title, summary, and category of approximately 2000 news articles.

In [10]:
# download reviews csv file
cvs_file_path = 'https://raw.githubusercontent.com/openai/openai-cookbook/main/examples/data/AG_news_samples.csv'
file_path = 'AG_news_samples.csv'

if not os.path.exists(file_path):
    wget.download(cvs_file_path, file_path)
    print('File downloaded successfully.')
else:
    print('File already exists in the local file system.')

File already exists in the local file system.


In [11]:
df = pd.read_csv('AG_news_samples.csv')
df

,title,description,label_int,label
0,World Briefings,BRITAIN: BLAIR WARNS OF CLIMATE THREAT Prime M...,1,World
1,Nvidia Puts a Firewall on a Motherboard (PC Wo...,PC World - Upcoming chip set will include buil...,4,Sci/Tech
2,"Olympic joy in Greek, Chinese press",Newspapers in Greece reflect a mixture of exhi...,2,Sports
3,U2 Can iPod with Pictures,"SAN JOSE, Calif. -- Apple Computer (Quote, Cha...",4,Sci/Tech
4,The Dream Factory,"Any product, any shape, any size -- manufactur...",4,Sci/Tech
...,...,...,...,...
1995,You Control: iTunes puts control in OS X menu ...,MacCentral - You Software Inc. announced on Tu...,4,Sci/Tech
1996,Argentina beat Italy for place in football final,Favourites Argentina beat Italy 3-0 this morni...,2,Sports
1997,NCAA case no worry for Spurrier,Shortly after Steve Spurrier arrived at Florid...,2,Sports
1998,Secret Service Busts Cyber Gangs,The US Secret Service Thursday announced arres...,4,Sci/Tech


In [12]:
data = df.to_dict(orient='records')
data[0]

{'title': 'World Briefings',
 'description': 'BRITAIN: BLAIR WARNS OF CLIMATE THREAT Prime Minister Tony Blair urged the international community to consider global warming a dire threat and agree on a plan of action to curb the  quot;alarming quot; growth of greenhouse gases.',
 'label_int': 1,
 'label': 'World'}

<div class="alert alert-block alert-warning">
    <b class="fa fa-solid fa-exclamation-circle"></b>
    <div>
        <p><b>Action Required</b></p>
        <p> If you have a Free Starter Workspace deployed already, select the database from drop-down menu at the top of this notebook. It updates the <tt>connection_url</tt> to connect to that database.</p>
    </div>
</div>

## Set up the database

Set up the SingleStoreDB database which will hold your data.

In [13]:
shared_tier_check = %sql show variables like 'is_shared_tier'
if not shared_tier_check or shared_tier_check[0][1] == 'OFF':
    %sql DROP DATABASE IF EXISTS news;
    %sql CREATE DATABASE news;

<div class="alert alert-block alert-warning">
    <b class="fa fa-solid fa-exclamation-circle"></b>
    <div>
        <p><b>Action Required</b></p>
        <p> Make sure to select a database from the drop-down menu at the top of this notebook. It updates the <tt>connection_url</tt>  to connect to that database.</p>
    </div>
</div>

In [14]:
%%sql
DROP TABLE IF EXISTS news_articles;
CREATE TABLE IF NOT EXISTS news_articles (
    title TEXT,
    description TEXT,
    genre TEXT,
    embedding BLOB,
    FULLTEXT (title, description)
);

++
||
++
++

### Get embeddings for every row based on the description column

In [15]:
# Will take around 3.5 minutes to get embeddings for all 2000 rows

descriptions = [row['description'] for row in data]
all_embeddings = model.encode(descriptions)
all_embeddings.shape

(2000, 768)

Merge embedding values into `data` rows.

In [16]:
for row, embedding in zip(data, all_embeddings):
    row['embedding'] = embedding

Here's an example of one row of the combined data.

In [17]:
data[0]

{'title': 'World Briefings',
 'description': 'BRITAIN: BLAIR WARNS OF CLIMATE THREAT Prime Minister Tony Blair urged the international community to consider global warming a dire threat and agree on a plan of action to curb the  quot;alarming quot; growth of greenhouse gases.',
 'label_int': 1,
 'label': 'World',
 'embedding': array([-1.42552713e-02, -1.03357071e-02,  1.25946105e-02,  8.40715785e-03,
        -6.92264410e-03, -8.77237227e-03, -5.38323671e-02,  1.95311196e-02,
         9.50564742e-02,  1.60899572e-02,  4.72200625e-02,  2.30231155e-02,
        -6.69442937e-02,  2.82599987e-03,  2.79738400e-02, -6.46088347e-02,
         5.52451760e-02, -4.02353071e-02, -2.22880822e-02, -1.65119395e-02,
         3.61824557e-02,  3.32110142e-03,  1.18329516e-02,  7.70277716e-03,
        -4.18954827e-02, -2.76368838e-02,  3.64982933e-02,  3.69321145e-02,
         5.97776957e-02,  8.05662386e-03,  3.38091105e-02, -1.52911590e-02,
         1.38111366e-02, -4.00905032e-03,  3.15332080e-08,  2.20

### Populate the database

In [18]:
%sql TRUNCATE TABLE news_articles;

import sqlalchemy as sa
from singlestoredb import create_engine

# Use create_table from singlestoredb since it uses the notebook connection URL
conn = create_engine().connect()

statement = sa.text('''
        INSERT INTO news_articles (
            title,
            description,
            genre,
            embedding
        )
        VALUES (
            :title,
            :description,
            :label,
            :embedding
        )
    ''')

conn.execute(statement, data)

## Semantic search

### Connect to OpenAI

In [19]:
import openai

EMBEDDING_MODEL = 'text-embedding-ada-002'
GPT_MODEL = 'gpt-3.5-turbo'

In [20]:
import os
import getpass
openai.api_key = os.getenv('OPENAI_API_KEY')




# openai.api_key = getpass.getpass('OpenAI API Key: ')

### Run semantic search and get scores

In [28]:
#search_query = 'Articles about Aussie captures'
search_query = 'Aussie'
search_embedding = model.encode(search_query)

query_statement = sa.text('''
    SELECT
        title,
        description,
        genre,
        DOT_PRODUCT(embedding, :embedding) AS score
    FROM news_articles
    ORDER BY score DESC
    LIMIT 10
    ''')


# # Create the SQL statement.
# query_statement = sa.text('''
#     SELECT
#         title
#     FROM news_articles

#     ''')

# Execute the SQL statement.
results = pd.DataFrame(conn.execute(query_statement, dict(embedding=search_embedding)))
print(results)

                                               title  \
0  All Australians accounted for in Iraq: Downer ...   
1                  A trio of television technologies   
2          National Foods posts increased net profit   
3                   Australia's leader wins 4th term   
4                    Cricket: Aussies dominate India   
5  Woman believed to be 1st to walk around Austra...   
6                    Australia clinches series sweep   
7           US buy spurs registrar #39;s share surge   
8                           Springboks targets scrum   
9                  Man tried for UK student's murder   

                                         description     genre     score  
0  AFP - Australia has accounted for all its nati...     World  0.305584  
1  AUSTRALIANS went into a television-buying fren...  Sci/Tech  0.245194  
2  Australia #39;s biggest supplier of fresh milk...  Business  0.233651  
3  SYDNEY -- Prime Minister John Howard of Austra...     World  0.216256  
4  Austr

## Hybrid search

This search finds the average of the score gotten from the semantic search and the score gotten from the key-word search and sorts the news articles by this combined score to perform an effective hybrid search.

In [30]:
hyb_query = 'Articles about Aussie captures'
hyb_embedding = model.encode(hyb_query)

# Create the SQL statement.
hyb_statement = sa.text('''
    SELECT
        title,
        description,
        genre,
        DOT_PRODUCT(embedding, :embedding) AS semantic_score,
        MATCH(title, description) AGAINST (:query) AS keyword_score,
        (semantic_score + keyword_score) / 2 AS combined_score
    FROM news_articles
    ORDER BY combined_score DESC
    LIMIT 10
    ''')

# Execute the SQL statement.
hyb_results = pd.DataFrame(conn.execute(hyb_statement, dict(embedding=hyb_embedding, query=hyb_query)))
hyb_results

,title,description,genre,semantic_score,keyword_score,combined_score
0,All Australians accounted for in Iraq: Downer ...,AFP - Australia has accounted for all its nati...,World,0.445396,0.0,0.222698
1,Cricket: Aussies dominate India,Australia tighten their grip on the third Test...,World,0.368577,0.0,0.184289
2,Man tried for UK student's murder,The trial of a man accused of murdering York b...,World,0.350485,0.0,0.175243
3,Ponting doesn #39;t think much of Kiwis or win...,RICKY PONTING believes the game #39;s watchers...,Sports,0.345483,0.0,0.172742
4,Hassan Body Found in Fallujah: Australian PM,Australia #39;s prime minister says a body fou...,World,0.341777,0.0,0.170889
5,Aussie alive after capture in Iraq,AUSTRALIAN journalist John Martinkus is lucky ...,World,0.334077,0.0,0.167039
6,A trio of television technologies,AUSTRALIANS went into a television-buying fren...,Sci/Tech,0.332006,0.0,0.166003
7,Australia PM Gets Down to Work on Fourth Term ...,Reuters - Australia's conservative Prime Minis...,World,0.324336,0.0,0.162168
8,"Police pull body of lost autistic man, 46, fro...","Canadian Press - OAKVILLE, Ont. (CP) - The bod...",World,0.322738,0.0,0.161369
9,Australia targeted for first time in Iraq car ...,Australian troops in Baghdad came under attack...,World,0.321895,0.0,0.160948


## Clean up

<div class="alert alert-block alert-warning">
    <b class="fa fa-solid fa-exclamation-circle"></b>
    <div>
        <p><b>Action Required</b></p>
        <p> If you created a new database in your Standard or Premium Workspace, you can drop the database by running the cell below. Note: this will not drop your database for Free Starter Workspaces. To drop a Free Starter Workspace, terminate the Workspace using the UI. </p>
    </div>
</div>

In [ ]:
shared_tier_check = %sql show variables like 'is_shared_tier'
if not shared_tier_check or shared_tier_check[0][1] == 'OFF':
    %sql DROP DATABASE IF EXISTS news;

<div id="singlestore-footer" style="background-color: rgba(194, 193, 199, 0.25); height:2px; margin-bottom:10px"></div>
<div><img src="https://raw.githubusercontent.com/singlestore-labs/spaces-notebooks/master/common/images/singlestore-logo-grey.png" style="padding: 0px; margin: 0px; height: 24px"/></div>